In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
!sudo apt-get update
!sudo apt-get install -y build-essential
!sudo apt-get install -y libopencv-dev
!sudo apt-get install -y cmake
!sudo apt-get install -y libprotobuf-dev libleveldb-dev libsnappy-dev libhdf5-serial-dev protobuf-compiler
!sudo apt-get install -y --no-install-recommends libboost-all-dev
!sudo apt-get install -y libgflags-dev libgoogle-glog-dev liblmdb-dev
!sudo apt-get install -y python3-setuptools python3-dev
!pip install opencv-python

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,108 kB]
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:8 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:9 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [8,472 kB]
Get:10 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [2,424 kB]
Hit:11 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [2,700 kB]
Hit:13 https://ppa.launchpadcontent.net

In [4]:
!git clone https://github.com/CMU-Perceptual-Computing-Lab/openpose.git
%cd openpose

Cloning into 'openpose'...
remote: Enumerating objects: 16156, done.
remote: Total 16156 (delta 0), reused 0 (delta 0), pack-reused 16156 (from 1)
Receiving objects: 100% (16156/16156), 84.46 MiB | 28.15 MiB/s, done.
Resolving deltas: 100% (11322/11322), done.
/content/openpose


In [5]:
!mkdir -p models
!cd models && wget -O /content/openpose/models/pose/coco/pose_iter_440000.caffemodel https://github.com/foss-for-synopsys-dwc-arc-processors/synopsys-caffe-models/raw/master/caffe_models/openpose/caffe_model/pose_iter_440000.caffemodel

--2024-11-14 11:18:50--  https://github.com/foss-for-synopsys-dwc-arc-processors/synopsys-caffe-models/raw/master/caffe_models/openpose/caffe_model/pose_iter_440000.caffemodel
Resolving github.com (github.com)... 140.82.114.3
Connecting to github.com (github.com)|140.82.114.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://media.githubusercontent.com/media/foss-for-synopsys-dwc-arc-processors/synopsys-caffe-models/master/caffe_models/openpose/caffe_model/pose_iter_440000.caffemodel [following]
--2024-11-14 11:18:50--  https://media.githubusercontent.com/media/foss-for-synopsys-dwc-arc-processors/synopsys-caffe-models/master/caffe_models/openpose/caffe_model/pose_iter_440000.caffemodel
Resolving media.githubusercontent.com (media.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to media.githubusercontent.com (media.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting respon

In [6]:
%cd /content/openpose
!mkdir build && cd build
!cmake ..
!make -j`nproc`

/content/openpose
CMake Warning:
  Ignoring extra path from command line:

   ".."


CMake Error: The source directory "/content" does not appear to contain CMakeLists.txt.
Specify --help for usage, or press the help button on the CMake GUI.
make: *** No targets specified and no makefile found.  Stop.


In [ ]:
import cv2
import json
import os
from google.colab.patches import cv2_imshow

protoFile = "/content/openpose/models/pose/coco/pose_deploy_linevec.prototxt"
weightsFile = "/content/openpose/models/pose/coco/pose_iter_440000.caffemodel"
net = cv2.dnn.readNetFromCaffe(protoFile, weightsFile)

inWidth = 368
inHeight = 368
thr = 0.1  # Confidence threshold for detected keypoints

input_folder = "/content/drive/MyDrive/Exercise Tracking/Extracted Exercise Frames"
output_folder = "/content/drive/MyDrive/Exercise Tracking/Annotated Exercise Frames"

for exercise in os.listdir(input_folder):
    exercise_path = os.path.join(input_folder, exercise)
    output_exercise_path = os.path.join(output_folder, exercise)
    os.makedirs(output_exercise_path, exist_ok=True)

    for frame_folder in os.listdir(exercise_path):
        frame_folder_path = os.path.join(exercise_path, frame_folder)
        output_frame_folder_path = os.path.join(output_exercise_path, frame_folder)
        os.makedirs(output_frame_folder_path, exist_ok=True)

        for frame_file in os.listdir(frame_folder_path):
            frame_path = os.path.join(frame_folder_path, frame_file)

            frame = cv2.imread(frame_path)
            if frame is None:
                print(f"Error loading image: {frame_path}")
                continue

            frameHeight, frameWidth = frame.shape[:2]
            inpBlob = cv2.dnn.blobFromImage(frame, 1.0 / 255, (inWidth, inHeight), (0, 0, 0), swapRB=False, crop=False)
            net.setInput(inpBlob)
            output = net.forward()

            BODY_PARTS = {
                "Nose": 0, "Neck": 1, "RShoulder": 2, "RElbow": 3, "RWrist": 4,
                "LShoulder": 5, "LElbow": 6, "LWrist": 7, "RHip": 8, "RKnee": 9,
                "RAnkle": 10, "LHip": 11, "LKnee": 12, "LAnkle": 13, "REye": 14,
                "LEye": 15, "REar": 16, "LEar": 17, "Background": 18
            }

            POSE_PAIRS = [
                ["Neck", "RShoulder"], ["Neck", "LShoulder"], ["RShoulder", "RElbow"],
                ["RElbow", "RWrist"], ["LShoulder", "LElbow"], ["LElbow", "LWrist"],
                ["Neck", "RHip"], ["RHip", "RKnee"], ["RKnee", "RAnkle"], ["Neck", "LHip"],
                ["LHip", "LKnee"], ["LKnee", "LAnkle"], ["Neck", "Nose"], ["Nose", "REye"],
                ["REye", "REar"], ["Nose", "LEye"], ["LEye", "LEar"]
            ]

            points = []
            keypoints_data = [] 

            for part, idx in BODY_PARTS.items():
                heatMap = output[0, idx, :, :]
                _, conf, _, point = cv2.minMaxLoc(heatMap)
                x = int((frameWidth * point[0]) / output.shape[3])
                y = int((frameHeight * point[1]) / output.shape[2])

                if conf > thr:
                    points.append((x, y))
                    keypoints_data.append({"part": part, "x": x, "y": y, "confidence": float(conf)})
                    cv2.circle(frame, (x, y), 5, (0, 255, 255), thickness=-1, lineType=cv2.FILLED)
                else:
                    points.append(None)
                    keypoints_data.append({"part": part, "x": 0, "y": 0, "confidence": 0.0})

            for pair in POSE_PAIRS:
                partFrom = pair[0]
                partTo = pair[1]
                idFrom = BODY_PARTS[partFrom]
                idTo = BODY_PARTS[partTo]

                if points[idFrom] and points[idTo]:
                    cv2.line(frame, points[idFrom], points[idTo], (0, 255, 0), 3)
                    cv2.circle(frame, points[idFrom], 5, (0, 0, 255), thickness=-1, lineType=cv2.FILLED)
                    cv2.circle(frame, points[idTo], 5, (0, 0, 255), thickness=-1, lineType=cv2.FILLED)

            output_frame_path = os.path.join(output_frame_folder_path, frame_file)
            cv2.imwrite(output_frame_path, frame)
            print(f"Annotated and saved: {output_frame_path}")

            json_output_path = os.path.join(output_frame_folder_path, f"{os.path.splitext(frame_file)[0]}.json")
            with open(json_output_path, "w") as json_file:
                json.dump({"keypoints": keypoints_data}, json_file, indent=4)
            print(f"Saved keypoints for frame {frame_file} to {json_output_path}")

Annotated and saved: /content/drive/MyDrive/Exercise Tracking/Annotated Exercise Frames/Head Turns/Head Turn 1/frame_0.jpg
Saved keypoints for frame frame_0.jpg to /content/drive/MyDrive/Exercise Tracking/Annotated Exercise Frames/Head Turns/Head Turn 1/frame_0.json
Annotated and saved: /content/drive/MyDrive/Exercise Tracking/Annotated Exercise Frames/Head Turns/Head Turn 1/frame_1.jpg
Saved keypoints for frame frame_1.jpg to /content/drive/MyDrive/Exercise Tracking/Annotated Exercise Frames/Head Turns/Head Turn 1/frame_1.json
Annotated and saved: /content/drive/MyDrive/Exercise Tracking/Annotated Exercise Frames/Head Turns/Head Turn 1/frame_2.jpg
Saved keypoints for frame frame_2.jpg to /content/drive/MyDrive/Exercise Tracking/Annotated Exercise Frames/Head Turns/Head Turn 1/frame_2.json
Annotated and saved: /content/drive/MyDrive/Exercise Tracking/Annotated Exercise Frames/Head Turns/Head Turn 1/frame_3.jpg
Saved keypoints for frame frame_3.jpg to /content/drive/MyDrive/Exercise Tra